In [16]:
from typing import List

In [17]:
def merge(intervals: List[List[int]]) -> List[List[int]]:
    """
    Merges all overlapping intervals.

    Args:
        intervals: List of [start, end] intervals

    Returns:
        List of merged non-overlapping intervals
    """
    temp = []
    start, end = intervals[0]
    
    if not intervals:
        return []
    
    intervals.sort() 


    for s,e in intervals[1:]:
        if s <= end:
            end = max(e, start)
        else:
            temp.append([start, end])
            start, end = s,e
    
    temp.append([start, end])
    return temp


In [18]:
assert merge([[1,4],[0,2],[3,5]]) == [[0,5]]

AssertionError: 

In [ ]:
assert merge([[1,3],[2,6],[8,10],[15,18]]) == [[1,6],[8,10],[15,18]]

In [ ]:
assert merge([[1,4],[4,5]]) == [[1,5]]

In [ ]:
assert merge([[1,4],[0,4]]) == [[0,4]]

In [ ]:
assert merge([]) == []

In [ ]:
assert merge([[5,7]]) == [[5,7]]

In [ ]:
### Second Test: 

In [19]:
from abc import ABC, abstractmethod
from datetime import datetime

class Product(ABC):
    _product_count = 0
    def __init__(self, name, price, stock):
        self._name = name
        self._price = price
        self._stock = stock
    
    @abstractmethod
    def get_product_info(self):
        pass
    
    @abstractmethod
    def get_category(self):
        pass
    
    def is_in_stock(self, quantity = 1):
        return self._stock >= quantity
        
    def reduce_stock(self, quantity): 
        if quantity > self._stock:
            raise ValueError("Not Enough stock to reduce")
        self._stock -= quantity
    def __str__(self):
        return f"{self._name} (${self._price}) - {self._stock} in stock"
    def __eq__(self, other):
        if not isinstance(other, Product):
            return NotImplemented
    
        return (self.get_category() == other.get_category() and self._name == other._name)
    
    def __hash__(self):
        return hash((self.__class__, self._name))
    

In [20]:
class ClothingProduct(Product):
    def __init__(self, name, price, stock, size, color):
        super().__init__(name, price, stock)
        self._size = size
        self._color = color
        
    def get_product_info(self):
        return f"{self._name} ({self._color}, size {self._size}) - ${self._price}"

    def get_category(self):
        return "Clothing"

In [21]:
class FoodProduct(Product):
    def __init__(self, name, price, stock, expiration_date):
        super().__init__(name, price, stock)
        self._expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")

    def is_expired(self):
        return datetime.now() > self._expiration_date

    def get_product_info(self):
        exp = self._expiration_date.strftime("%Y-%m-%d")
        return f"{self._name} (exp: {exp}) - ${self._price}"

    def get_category(self):
        return "Food"

In [22]:
class ShoppingCart:
    def __init__(self):
        self._items = {}
    
    def add_item(self, product, quantity):
        if not product.is_in_stock(quantity):
            raise ValueError(f"Not Enough Stock")
        self._items[product] = quantity + self._items.get(product, 0)
    
    def remove_item(self, product):
        if product in self._items:
            del self._items[product]
        
    def get_total(self):
        return sum(item._price * count for item, count in self._items.items())
    
    def get_items(self):
        return [(item, count) for item, count in self._items.items()]
    
    def checkout(self):
        print("\n=== Order Summary ===")
        for item, count in self._items.items():
            print(f"{item.get_product_info()} x {count} = ${item._price * count}")
        print(f"Total: ${self.get_total()}")

        print("\nUpdating inventory...")
        for item, count in self._items.items():
            item.reduce_stock(count)
        print("Inventory updated successfully.")

        self.clear_cart()
    
    def clear_cart(self):
        self._items.clear()
        print("Cleared")
    
    def __len__(self):
        return sum(self._items.values())

In [23]:
shirt = ClothingProduct("T-Shirt", 25.0, 10, "M", "Blue")
jeans = ClothingProduct("Jeans", 60.0, 5, "L", "Black")
apple = FoodProduct("Apple", 1.5, 20, "2025-12-31")

cart = ShoppingCart()

cart.add_item(shirt, 2)
cart.add_item(apple, 5)
cart.add_item(jeans, 1)

print("\nItems in cart:", cart.get_items())
print("Total items:", len(cart))
print(f"Total amount: ${cart.get_total():.2f}")

cart.checkout()

print("\n--- Remaining Stock ---")
for product in [shirt, jeans, apple]:
    print(product)


Items in cart: [(<__main__.ClothingProduct object at 0x1182f1eb0>, 2), (<__main__.FoodProduct object at 0x1182f0200>, 5), (<__main__.ClothingProduct object at 0x1182f0c50>, 1)]
Total items: 8
Total amount: $117.50

=== Order Summary ===
T-Shirt (Blue, size M) - $25.0 x 2 = $50.0
Apple (exp: 2025-12-31) - $1.5 x 5 = $7.5
Jeans (Black, size L) - $60.0 x 1 = $60.0
Total: $117.5

Updating inventory...
Inventory updated successfully.
Cleared

--- Remaining Stock ---
T-Shirt ($25.0) - 8 in stock
Jeans ($60.0) - 4 in stock
Apple ($1.5) - 15 in stock
